## Тестовое задание Авито

Процесс обучения происходил с помощью вычислительных возможностей kaggle (GPU P 100)

In [1]:
!pip install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2da7a7244a7cc0948c2c75684b441b24c507d440909bbd9a4622232d23defa38
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesi

### Импортируем зависимости и фиксируем сиды

In [2]:
from transformers import AutoTokenizer
from datasets import Dataset, Features, Value, ClassLabel, Sequence
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import random
import torch

Попытка обучить "русскую" модель ruBert от DeepPavlov с токенизатором WordPiece

In [ ]:
SEED = 42
CPU_COUNT = os.cpu_count()
# необходимо поменять пути в kaggle для повторения эскперимента
DATA_PATH = "/kaggle/input/dfghjklgfhj/rec_aaa_title_desc.pq"
TEST_DATA_PATH = "/kaggle/input/test-dataset-avito-internship-v1/dataset_1937770_3.txt"

In [4]:
def set_seed(seed: int):
    """Фиксирует все сиды для воспроизводимости"""
    # Python
    random.seed(seed)
    
    # Numpy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # если используется несколько GPU
    
    # Установки для CuDNN (может снизить производительность)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Для трансформеров
    try:
        import transformers
        transformers.set_seed(seed)
    except:
        pass

# Установите сид перед началом обучения
set_seed(SEED)

2025-09-22 07:46:02.270321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758527162.461233      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758527162.516618      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def setup_environment_for_reproducibility(seed: int):
    """Настройка окружения для максимальной воспроизводимости"""
    # Установка сидов
    set_seed(seed)
    
    # Переменные окружения
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    
    # Дополнительные настройки PyTorch
    torch.use_deterministic_algorithms(True, warn_only=True)

# Использование
setup_environment_for_reproducibility(SEED)

### Подготовка данных

В качестве данных были взяты описания объявлений с платформы Авито

In [6]:
# Загрузка токенизатора нужной модели
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Читаем данные и создаем датасет
df = pd.read_parquet(DATA_PATH)

features = Features({
    "descriptions": Value("string"),
})
descriptions_dataset = Dataset.from_dict({"descriptions": df['DescriptionRu'].dropna().tolist()[:140000] + df['Title'].dropna().tolist()[:70000]}, features=features)

In [ ]:
# Словари для лэйблов классификации
label2id = {"B": 0, "I": 1}
id2label = {0: "B", 1: "I"}

Следующая фукнция отличается от функции для CANINE

In [ ]:
def tokenized_and_labels(examples):
    """Функция для токенизации описаний и выставлении лэйблов для пробелов"""
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    
    for desc in examples["descriptions"]:
        # Предобработка текста
        original_text = desc.replace('\n', ' ').strip()
        words = original_text.split(" ")  # Разбиваем на слова по пробелам

        # 1. Создаем выравнивание: для каждого символа в тексте БЕЗ пробелов определяем, с какого слова он пришел
        fused_text = original_text.replace(" ", "")
        char_to_word = []
        for word in words:
            # Для каждого символа в слове (кроме последнего) добавляем индекс слова
            for _ in word:
                char_to_word.append(word)
        
        # 2. Получаем разметку на уровне символов (БЕЗ пробелов)
        char_labels = []
        current_word = None
        for word in char_to_word:
            if word != current_word:
                # Новое слово -> метка 'B'
                char_labels.append("B")
                current_word = word
            else:
                # Продолжение текущего слова -> метка 'I'
                char_labels.append("I")

        # 3. Токенизируем текст БЕЗ пробелов и получаем offset mapping
        tokenized = tokenizer(
            fused_text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_offsets_mapping=True
        )

        # 4. Сопоставляем токены и символьные метки
        labels = []
        offsets = tokenized["offset_mapping"]
        
        for i, (start, end) in enumerate(offsets):
            # Обрабатываем специальные токены [CLS], [SEP], [PAD]
            if start == end:
                labels.append(-100)
                continue
                
            # Получаем метку первого символа в токене
            first_char_label = char_labels[start]
            labels.append(label2id[first_char_label])
            
            # ВАЖНО: Если токен покрывает несколько символов, мы игнорируем возможные изменения метки внутри токена

        input_ids_list.append(tokenized['input_ids'])
        attention_mask_list.append(tokenized['attention_mask'])
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "labels": labels_list,
    }

In [10]:
dataset = descriptions_dataset.map(tokenized_and_labels, batched=True, batch_size=16, remove_columns=["descriptions"], num_proc=CPU_COUNT)

Map (num_proc=4):   0%|          | 0/210000 [00:00<?, ? examples/s]

In [11]:
# Разделяем на train/validation
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset_dict = {
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']
}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

# загружаем модель с hugginface
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    """Функция для получения метрик precision recall f1 accuracy во время обучения"""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    run_name="train_bert",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none",
    seed=SEED,
    dataloader_drop_last=True,  # Избегает частичных батчей
    dataloader_pin_memory=False,  # Может влиять на воспроизводимость
)

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Memory Efficient attention defaults to a non-deterministic algorithm. To explicitly enable determinism call torch.use_deterministic_algorithms(True, warn_only=False). (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/attention_backward.cu:683.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.103000,0.097750,0.878941,0.891267,0.885061,0.965652
2,0.092600,0.093740,0.885509,0.894010,0.889739,0.967273


TrainOutput(global_step=10500, training_loss=0.10606013961065383, metrics={'train_runtime': 23671.8284, 'train_samples_per_second': 14.194, 'train_steps_per_second': 0.444, 'total_flos': 8.7795710263296e+16, 'train_loss': 0.10606013961065383, 'epoch': 2.0})

In [ ]:
# сохраняем модель
trainer.save_model("./bert-space-restoration-final")
tokenizer.save_pretrained("./bert-space-restoration-final")

('./bert-space-restoration-final/tokenizer_config.json',
 './bert-space-restoration-final/special_tokens_map.json',
 './bert-space-restoration-final/vocab.txt',
 './bert-space-restoration-final/added_tokens.json',
 './bert-space-restoration-final/tokenizer.json')

In [ ]:
from transformers import pipeline

restore_pipe = pipeline(
    "token-classification", 
    model="./bert-space-restoration-final", 
    tokenizer=tokenizer,
    aggregation_strategy="none",  # Отключаем встроенную агрегацию
    device=0 if torch.cuda.is_available() else -1
)

def restore_spaces_canine(text):
    # Получаем предсказания для каждого токена
    predictions = restore_pipe(text)
    
    # Создаем список символов результата
    result_chars = []
    
    for i, pred in enumerate(predictions):
        current_token = pred['word']
        current_entity = pred['entity']
        
        # Пропускаем специальные токены
        if current_token in ['[CLS]', '[SEP]']:
            continue
            
        # Определяем, нужно ли добавить пробел перед текущим токеном
        if current_entity == 'B' and i != 0:
            result_chars.append(' ')
            
        # Добавляем текущий токен к результату (убираем ## для subword-токенов)
        if current_token.startswith('##'):
            result_chars.append(current_token[2:])
        else:
            result_chars.append(current_token)
    
    # Собираем финальную строку
    result_text = ''.join(result_chars)
    
    return result_text

# Пример использования
test_text = "книгавхорошемсостоянии"
restored_text = restore_spaces_canine(test_text)
print(f"Input: {test_text}")
print(f"Output: {restored_text}")

NameError: name 'pipeline' is not defined

### Блок для скачивания модели с kaggle

In [ ]:
!zip -r final_model.zip /kaggle/working/bert-space-restoration-final

In [ ]:
%cd /kaggle/working

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink('final_model.zip')

### Получаем предикт на тестовых данных и сохраняем рузультат

In [ ]:
def preprocess_file(file_path):
    """Функция для чтения файла с тестовыми данными. (запятые используется и как сепаратор для данных и в самих данных)"""
    cleaned_lines = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) > 2:
                cleaned_line = [parts[0], ','.join(parts[1:])]
                cleaned_lines.append(cleaned_line)
            else:
                cleaned_lines.append(parts)
    return cleaned_lines

cleaned_data = preprocess_file(TEST_DATA_PATH)
test_df = pd.DataFrame(cleaned_data[1:])

In [ ]:
restore_pipe = pipeline(
    "token-classification", 
    model="./bert-space-restoration-final", 
    tokenizer=tokenizer,
    aggregation_strategy="none",
    device=0 if torch.cuda.is_available() else -1
)

def get_space_positions(text):
    # Удаляем пробелы из текста, так как модель обучена на тексте без пробелов
    text_without_spaces = text.replace(" ", "")
    predictions = restore_pipe(text_without_spaces)
    
    space_positions = []
    current_position = 0
    
    for i, pred in enumerate(predictions):
        current_token = pred['word']
        current_entity = pred['entity']
        
        # Пропускаем специальные токены
        if current_token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
            
        # Убираем префикс ## для subword-токенов
        if current_token.startswith('##'):
            clean_token = current_token[2:]
        else:
            clean_token = current_token
        
        # Если это начало слова (кроме первого токена), добавляем позицию пробела
        if current_entity == 'B' and i > 0:
            space_positions.append(current_position)
        
        # Увеличиваем текущую позицию на длину очищенного токена
        current_position += len(clean_token)
    
    return space_positions

def process_dataframe(df):
    results = []
    
    for index, row in df.iterrows():
        text_id = row[0]  # ID из первой колонки
        text = row[1]     # Текст из второй колонки
        
        try:
            space_positions = get_space_positions(text)
            results.append({
                'id': text_id,
                'predicted_positions': space_positions
            })
        except Exception as e:
            print(f"Error processing ID {text_id}: {e}")
            results.append({
                'id': text_id,
                'predicted_positions': []
            })
    
    return pd.DataFrame(results)

# Применяем функцию к test_df
result_df = process_dataframe(test_df)

# Выводим результат
print(result_df.head())

# Сохраняем при необходимости
result_df.to_csv('predicted_space_positions.csv', index=False, encoding='utf-8')